In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn import Flatten
import torchvision 
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset

In [2]:
cwd = os.getcwd()
cwd

'/home/aarso2/BPI/Projeto_Semana6'

In [67]:
class DatasetEmotions(torch.utils.data.Dataset):
    def __init__(self,path, transforms = None):
        super().__init__()
        self.path = path
        self.images = glob.glob(os.path.join(self.path,"*_oneface/*"))
        self.labels = {"anger":0,"joy":1,"neutral":2}
        self.transforms = transforms
    def __getitem__(self,idx):
        image = cv2.imread(self.images[idx])
        if self.transforms != None: 
            image = self.transforms(image)
        return image, self.labels[self.images[idx].split("/")[-2].replace("_oneface", "")]

    def __len__(self):
        return len(self.images)

In [76]:
# Define your data folder and optional image transformations
train_path = cwd+'/BPI-MELD-main/train/train_splits_images/'
dev_path = cwd+'/BPI-MELD-main/dev/dev_splits_images/'

transforming = T.Compose([T.Resize((112, 112)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

# Create the custom dataset and data loader
train_dataset = DatasetEmotions(train_path, transforms=transforming)
dev_dataset = DatasetEmotions(dev_path, transforms=transforming)
train_data = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_data = DataLoader(dev_dataset, batch_size=32, shuffle=True)

In [77]:
total_samples_train = len(train_data) * train_data.batch_size
total_samples_dev = len(dev_data) * dev_data.batch_size
print(f"Total de amostras no conjunto de dados de treino: {total_samples_train}")
print(f"Total de amostras no conjunto de dados de teste: {total_samples_dev}")

Total de amostras no conjunto de dados de treino: 5760
Total de amostras no conjunto de dados de teste: 672


In [78]:
# Define the neural network model
class EmotionRecognitionNet(nn.Module):
    def __init__(self, num_classes):
        super(EmotionRecognitionNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * (112 // 8) * (112 // 8), 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the output for fully connected layers
        x = self.fc_layers(x)
        return x

In [ ]:
# Number of emotion classes
num_classes = 3

# Initialize the model, loss function, and optimizer
model = EmotionRecognitionNet(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (assuming you have train_data DataLoader)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch_idx, (inputs, targets) in enumerate(train_data):
        # Move inputs and targets to the device (CPU or GPU)
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Print training progress
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_data)}], Loss: {loss.item():.4f}')

    # Validation (assuming you have dev_data DataLoader)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, targets in dev_data:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        accuracy = 100 * correct / total
        print(f'Validation Accuracy: {accuracy:.2f}%')

print('Training finished.')